see: https://towardsdatascience.com/how-to-build-your-own-chatbot-using-deep-learning-bb41f970e281

In [14]:
%%writefile requirements.txt
tensorflow==2.3.1
nltk==3.5
colorama==0.4.3
numpy==1.18.5
scikit_learn==0.23.2
Flask==1.1.2
mlflow

Overwriting requirements.txt


In [15]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 13.9MB 241kB/s 
     |████████████████████████████████| 6.0MB 39.0MB/s 
     |████████████████████████████████| 327kB 32.3MB/s 
     |████████████████████████████████| 153kB 15.6MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 1.1MB 39.1MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 163kB 43.1MB/s 
     |████████████████████████████████| 2.6MB 39.9MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 122kB 44.9MB/s 
     |████████████████████████████████| 204kB 42.5MB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.13-cp36-cp36m-linux_x86_64.whl size=1217166 sha256=cb3051de127e3ac26bccf0b7dddda5926a50828d6979302ed73432f46c8b6da3

In [4]:
%%writefile intents.json
{"intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?" ],
     "responses": ["I.m Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["what is your name", "what should I call you", "whats your name?"],
    "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
    "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "createaccount",
    "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
    "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
    },
    {"tag": "complaint",
    "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
    "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
    }
]
}

Writing intents.json


In [16]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import mlflow

###Mlflow config

In [21]:
#Mlflow settings
MLFLOW_URL='http://mlops.getmytrainer.net:5001'
EXPERIMENT='npl/chatbot'
EPOCHS=500
VERSION=1
LOSS='sparse_categorical_crossentropy'
OPTIMIZER='adam'

#Set tags
tags={}
tags['TYPE']='IMAGE_RECOGNITION'
tags['TRANSFER_LEARNING']='NO'
tags['TFHUB']='NO'

SAVE_PATH='saved_model'
%mkdir saved_model

#set MLflow server 
mlflow.set_tracking_uri(MLFLOW_URL)
#Set experiment
if mlflow.get_experiment_by_name(EXPERIMENT) != None:
    exp_id = mlflow.set_experiment(EXPERIMENT)
else: 
    exp_id = mlflow.create_experiment(EXPERIMENT)

#Close active runs
if mlflow.active_run():
    mlflow.end_run()

mkdir: cannot create directory ‘saved_model’: File exists


### Data Preparation

In [6]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [7]:
print("training sentences: ", len(training_sentences)) #holds all the training data (which are the sample messages in each intent category)
print("training labels: ", len(training_labels)) #holds all the target labels correspond to each training data
print("labels: ", len(labels))
print("responses: ", len(responses))

training sentences:  33
training labels:  33
labels:  8
responses:  8


In [8]:
#convert the target labels into a model understandable form
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [9]:
#vectorize our text data corpus by using the “Tokenizer” class and it allows us to limit our vocabulary size up to some defined number.
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len) #used to make all the training text sequences into the same size

### Model Training

In [20]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=LOSS, 
              optimizer=OPTIMIZER, metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 136       
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
_________________________________________________________________


In [25]:
with mlflow.start_run(run_id=None, experiment_id=exp_id, run_name=None, nested=False):
  
  history = model.fit(padded_sequences, np.array(training_labels), epochs=EPOCHS)

  hist=history.history

  #mlflow autolog
  #mlflow.tensorflow.autolog()

  #Set tags
  mlflow.set_tags(tags)

  #mlflow logging
  # log parameters
  #mlflow.log_param("hidden_layers", args.hidden_layers)
  #mlflow.log_param("output", args.output)
  mlflow.log_param("epochs", EPOCHS)
  mlflow.log_param("loss_function", LOSS)
  mlflow.log_param("optimizer", OPTIMIZER)

  # log metrics
  for epoch in range(0, EPOCHS):
    mlflow.log_metric("accuracy", hist['accuracy'][epoch])
    mlflow.log_metric("loss",  hist['loss'][epoch])
    #mlflow.log_metric("val_accuracy",  hist['val_accuracy'][epoch])
    #mlflow.log_metric("val_loss",  hist['val_loss'][epoch])
  
  #log model
  #model.save(os.path.join(BASE_DIR, "models", "{}.h5".format(int(t)))) #HDF5 format
  tf.saved_model.save(model, SAVE_PATH) #SavedModel format
  #mlflow.tensorflow.log_model(model, 'model') #TODO fix
  
  # log artifacts (matplotlib images for loss/accuracy)
  #mlflow.log_artifacts(image_dir)

  mlflow.end_run()

Epoch 1/500
2/2 [==============================] - 0s 5ms/step - loss: 0.0645 - accuracy: 1.0000
Epoch 2/500
2/2 [==============================] - 0s 3ms/step - loss: 0.0649 - accuracy: 1.0000
Epoch 3/500
2/2 [==============================] - 0s 4ms/step - loss: 0.0651 - accuracy: 1.0000
Epoch 4/500
2/2 [==============================] - 0s 5ms/step - loss: 0.0658 - accuracy: 1.0000
Epoch 5/500
2/2 [==============================] - 0s 4ms/step - loss: 0.0659 - accuracy: 1.0000
Epoch 6/500
2/2 [==============================] - 0s 4ms/step - loss: 0.0651 - accuracy: 1.0000
Epoch 7/500
2/2 [==============================] - 0s 4ms/step - loss: 0.0645 - accuracy: 1.0000
Epoch 8/500
2/2 [==============================] - 0s 3ms/step - loss: 0.0639 - accuracy: 1.0000
Epoch 9/500
2/2 [==============================] - 0s 4ms/step - loss: 0.0630 - accuracy: 1.0000
Epoch 10/500
2/2 [==============================] - 0s 3ms/step - loss: 0.0618 - accuracy: 1.0000
Epoch 11/500
2/2 [===========

In [26]:
import pickle

%mkdir -p utils

# to save the fitted tokenizer
with open('utils/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('utils/label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('saved_model')

    # load tokenizer object
    with open('utils/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('utils/label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: Ros
ChatBot: Have a nice day
User: hello
ChatBot: Hi
User: ciao
ChatBot: Bye! Come back again
User: greetings
ChatBot: See you later
User: what's your name?
ChatBot: Just call me as Joana
User: Joana?
ChatBot: Bye! Come back again
User: who are you?
ChatBot: I.m Joana, your bot assistant
User: what's the meaning of the life?
ChatBot: See you later
User: 

KeyboardInterrupt: ignored